In [ ]:
from ib_insync import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import logging

class TradingBot:
    def __init__(self, symbol="SPY", exchange="SMART", currency="USD"):
        """
        Initialize the trading bot with connection to Interactive Brokers
        """
        self.ib = IB()
        self.symbol = symbol
        self.exchange = exchange
        self.currency = currency
        self.positions = {}
        self.bars_data = pd.DataFrame()
        self.is_running = False
        
        # Setup logging
        logging.basicConfig(
            filename=f'trading_bot_{datetime.now().strftime("%Y%m%d")}.log',
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        
    def connect(self, port=7497, client_id=1):
        """
        Connect to Interactive Brokers TWS or Gateway
        """
        try:
            self.ib.connect('127.0.0.1', port, clientId=client_id)
            logging.info("Successfully connected to Interactive Brokers")
            return True
        except Exception as e:
            logging.error(f"Failed to connect to Interactive Brokers: {str(e)}")
            return False
            
    def get_contract(self):
        """
        Create contract object for the specified symbol
        """
        contract = Future(self.symbol, exchange=self.exchange, currency=self.currency)
        return contract

    def fetch_historical_data(self, duration='2 D', bar_size='5 mins'):
        """
        Fetch historical data for the specified duration and bar size
        """
        contract = self.get_contract()
        bars = self.ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr=duration,
            barSizeSetting=bar_size,
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1
        )
        return bars

    def calculate_signals(self, strategy='SMA'):
        """
        Calculate trading signals based on the specified strategy
        """
        if len(self.bars_data) < 20:
            return None
            
        if strategy == 'SMA':
            # Simple Moving Average Crossover Strategy
            self.bars_data['SMA20'] = self.bars_data['close'].rolling(window=20).mean()
            self.bars_data['SMA50'] = self.bars_data['close'].rolling(window=50).mean()
            
            # Generate signals
            if self.bars_data['SMA20'].iloc[-1] > self.bars_data['SMA50'].iloc[-1] and \
               self.bars_data['SMA20'].iloc[-2] <= self.bars_data['SMA50'].iloc[-2]:
                return 'BUY'
            elif self.bars_data['SMA20'].iloc[-1] < self.bars_data['SMA50'].iloc[-1] and \
                 self.bars_data['SMA20'].iloc[-2] >= self.bars_data['SMA50'].iloc[-2]:
                return 'SELL'
        
        return None

    def place_order(self, action, quantity=1):
        """
        Place a market order
        """
        contract = self.get_contract()
        order = MarketOrder(action, quantity)
        trade = self.ib.placeOrder(contract, order)
        
        logging.info(f"Placed {action} order for {quantity} contracts of {self.symbol}")
        return trade

    def manage_position(self, signal):
        """
        Manage positions based on trading signals
        """
        current_position = self.get_position()
        
        if signal == 'BUY' and current_position <= 0:
            # Close any existing short position
            if current_position < 0:
                self.place_order('BUY', abs(current_position))
            # Open new long position
            self.place_order('BUY', 1)
            
        elif signal == 'SELL' and current_position >= 0:
            # Close any existing long position
            if current_position > 0:
                self.place_order('SELL', current_position)
            # Open new short position
            self.place_order('SELL', 1)

    def get_position(self):
        """
        Get current position for the symbol
        """
        positions = self.ib.positions()
        for position in positions:
            if position.contract.symbol == self.symbol:
                return position.position
        return 0

    def on_bar_update(self, bars, has_new_bar):
        """
        Callback function for real-time bar updates
        """
        if has_new_bar:
            # Update bars data
            bar_dict = {
                'date': bars[-1].date,
                'open': bars[-1].open,
                'high': bars[-1].high,
                'low': bars[-1].low,
                'close': bars[-1].close,
                'volume': bars[-1].volume
            }
            self.bars_data = self.bars_data.append(bar_dict, ignore_index=True)
            
            # Calculate signals and manage positions
            signal = self.calculate_signals()
            if signal:
                self.manage_position(signal)
            
            logging.info(f"New bar processed - Close: {bars[-1].close}, Signal: {signal}")

    def run(self, strategy='SMA'):
        """
        Main method to run the trading bot
        """
        if not self.connect():
            return
            
        try:
            self.is_running = True
            contract = self.get_contract()
            
            # Get historical data
            historical_bars = self.fetch_historical_data()
            self.bars_data = pd.DataFrame([{
                'date': bar.date,
                'open': bar.open,
                'high': bar.high,
                'low': bar.low,
                'close': bar.close,
                'volume': bar.volume
            } for bar in historical_bars])
            
            # Subscribe to real-time bars
            self.ib.reqRealTimeBars(contract, 5, 'TRADES', False)
            
            # Main loop
            while self.is_running:
                self.ib.sleep(1)
                self.ib.run()
                
        except Exception as e:
            logging.error(f"Error in main loop: {str(e)}")
            self.is_running = False
        finally:
            self.ib.disconnect()

if __name__ == "__main__":
    # Initialize and run the trading bot
    bot = TradingBot(symbol="SPY", exchange="SMART")
    bot.run(strategy='SMA')